# Predict ED_dispo with result from Lbl2TransformerVec and without

For architecture and weight initialization, we will compare:

• BERT (baseline)

• BioBERT

• Clinical BioBERT

• BlueBERT

• XLNet (baseline)

• Clinical XLnet

In [ ]:
#!pip install tensorflow_hub

In [ ]:
#!pip install tensorflow-text --no-dependencies

In [ ]:
import pandas as pd
import numpy as np
import gc
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, auc
import re
import tensorflow as tf
from transformers import TFXLNetModel, XLNetTokenizer
from sklearn.utils.fixes import loguniform
from sklearn.model_selection import  train_test_split
from sklearn.model_selection import ParameterSampler
import tensorflow_hub as hub
import json
import csv 

import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

import mlflow
import mlflow.keras
import tensorflow_text as text
import seaborn as sns

In [ ]:
#!pip install mlflow
#!databricks configure --host https://community.cloud.databricks.com/

In [ ]:
%env  DATABRICKS_USERNAME= 'karengar@stanford.edu' 
%env  DATABRICKS_PASSWORD = 'Projectcs224*'

 - Import data

In [ ]:
data_test = pd.read_csv('/root/data/ed_test.csv')
data_train = pd.read_csv('/root/data/ed_train.csv')
data_val = pd.read_csv('/root/data/ed_val.csv')
data_val.head()

In [ ]:
data_train.groupby('ED_dispo').size()

In [ ]:
(data_train.groupby('ED_dispo').size()/data_train.groupby('ED_dispo').size().sum()) *100

In [ ]:
data_test.groupby('ED_dispo').size()

In [ ]:
(data_test.groupby('ED_dispo').size()/data_test.groupby('ED_dispo').size().sum())*100

In [ ]:
data_val.groupby('ED_dispo').size().sum()

In [ ]:
(data_val.groupby('ED_dispo').size()/data_val.groupby('ED_dispo').size().sum()) *100

In [ ]:
#result_l2v = pd.read_csv('/root/Project-CS224N-ED-Disposition/result_l2v_v1.csv') 
#result_l2v.head()

In [ ]:
def plot_roc_curve(y_proba, y_test, labels, mlflow, split_type):
    # Plots the Probability Distributions and the ROC Curves One vs Rest
    fig = plt.figure(figsize = (17, 8))
    bins = [i/20 for i in range(20)] + [1]
    roc_auc_ovr = {}
    for i in range(len(labels)):
        # Gets the class
        c = labels[i]

        # Prepares an auxiliar dataframe to help with the plots
        df_aux = pd.DataFrame()
        df_aux['class'] = [1 if y == c else 0 for y in y_test]
        df_aux['prob'] = y_proba[:, i]
        df_aux = df_aux.reset_index(drop = True)

        # Plots the probability distribution for the class and the rest
        fig_upper = plt.subplot(2, 4, i+1)
        x = df_aux[df_aux['class'] ==0]
        plt.hist(x['prob'], density=True, label='Rest')
        y = df_aux[df_aux['class'] ==1]
        plt.hist(y['prob'], density=True, label=f" {c}", bins = bins)
        plt.title(c)
        plt.legend(loc='upper right')
        plt.xlabel(f"P(x = {c})")
        #plt.show()
        #mlflow.log_figure(fig, f"Histogram: {c}.png") 


        # Calculates the ROC Coordinates and plots the ROC Curves
        fpr, tpr, thresholds = roc_curve(df_aux['class'], df_aux['prob'], pos_label=1)
        mauc = auc(fpr, tpr)
        fig_bottom = plt.subplot(2, 4, i+5)
        plt.title( f"ROC: {c}")
        plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % mauc)
        plt.legend(loc = 'lower right')
        plt.plot([0, 1], [0, 1],'r--')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        #plt.show()
        mlflow.log_metric("auc_" + split_type + "_"+  c, mauc) 

    plt.tight_layout()
    mlflow.log_figure(fig, "Hist_and_ROC_" + c+ "_"+ split_type + ".png") 
    
def evaluate(y_test, y_pred, mlflow,  split_type):
    """
    Evaluation function. For each of the text in evaluation data, it reads the score from
    the predictions made. And based on this, it calculates the values of
    True positive, True negative, False positive, and False negative.

    :param y_test: true labels
    :param y_pred: predicted labels
    :param labels: list of possible labels
    :return: evaluation metrics for classification like, precision, recall, and f_score
    """
     
    b = np.zeros_like(y_pred)
    b[np.arange(len(y_pred)), y_pred.argmax(1)] = 1
    y_proba = y_pred

    y_pred = pd.DataFrame(b, columns = list(y_test.columns))
    y_pred = y_pred.idxmax(axis=1)
    y_test = y_test.idxmax(axis=1)

    labels = list(y_test.unique())
    labels = sorted(labels)

    plot_roc_curve(y_proba, y_test, labels, mlflow, split_type)

    
    confusion = confusion_matrix(y_test, y_pred)

    # importing accuracy_score, precision_score, recall_score, f1_score
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average= None)
    recall = recall_score(y_test, y_pred, average= None)
    f1score = f1_score(y_test, y_pred, average= None)


    report = classification_report(y_test, y_pred, target_names = labels)
    print(report)
    f_name = 'report_ '+ split_type +'.yaml'
    mlflow.log_dict(report, f_name)
    #mlflow.log_metric("auc_" + split_type, mauc) 
    mlflow.log_metric("accuracy_"+ split_type, accuracy)
   

    return accuracy, precision, recall, f1score




def mlflow_log_parameters(parameter):
    # Log parameters
    mlflow.log_param("dropout", parameter['dropout'])
    mlflow.log_param("learning_rate", parameter['learning_rate'])
    mlflow.log_param("epochs", parameter['epochs'])
    mlflow.log_param("batch_size", parameter['batch_size'])


def get_inputs(impresions, tokenizer, max_len=120):
    """ Gets tensors from text using the tokenizer provided"""
    inps = [tokenizer.encode_plus(t, max_length=max_len, pad_to_max_length=True, add_special_tokens=True) for t in impresions]
    inp_tok = np.array([a['input_ids'] for a in inps])
    ids = np.array([a['attention_mask'] for a in inps])
    segments = np.array([a['token_type_ids'] for a in inps])
    return inp_tok, ids, segments

def create_bert(learning_rate=1e-3, dropout=0.1):
    # Input layer
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Text')
    text_input.trainable = False
    
    # Bert encoding
    bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3", name = "Bert_preprocessing")
    # Freeze the bert_preprocess layer
    bert_preprocess.trainable = False
    
    bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_L-24_H-1024_A-16/4", name = "Bert_encoding")
    # Freeze the bert_encoder layer
    bert_encoder.trainable = False
    
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)
    
    # Classification layer with dropout for regularization
    layer1 = tf.keras.layers.Dropout(dropout, name='Dropout')(outputs['pooled_output'])
    # Freeze the layer1
    layer1.trainable = False

    layer2 = tf.keras.layers.Dense(4, activation='softmax', name='outputs')(layer1)
    layer2.trainable = False
    # Compile model
    model = tf.keras.Model(inputs=[text_input], outputs=[layer2])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', 
                   metrics=[tf.keras.metrics.AUC(), 'accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    return model


def training_test_mae( mlflow, history):
    # Plot training and test loss at each epoch 
    fig, ax = plt.subplots()
    ax.plot(history.history['accuracy'], label='Training acc')
    #plt.plot(history.history['val_accuracy'], label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('accuracy')
    plt.legend()
    plt.show()
    mlflow.log_figure(fig, "training_validation_accuracy.png") 

    fig, ax = plt.subplots()
    plt.plot(history.history['loss'])
    #plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    #plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    mlflow.log_figure(fig, "training_validation_loss.png") 


# Scenario 1

- Without using results from l2v

In [ ]:
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/karengar@stanford.edu/BERT_l2v")

In [ ]:
#data[list(result_l2v.iloc[:, 3:].columns)] = result_l2v.iloc[:, 3:]
#data.head()

In [ ]:
X_train = data_train.drop(['Unnamed: 0', 'ED_dispo', 'CSN', 'Rads_order_ID'], axis=1)
X_test =  data_test.drop(['Unnamed: 0', 'ED_dispo', 'CSN', 'Rads_order_ID'], axis=1)
X_val = data_val.drop(['Unnamed: 0', 'ED_dispo', 'CSN', 'Rads_order_ID'], axis=1)

# Assigning numerical values and storing in another column
y_train =  pd.get_dummies(data_train['ED_dispo'])
y_test = pd.get_dummies(data_test['ED_dispo'])[list(y_train.columns)]
y_val =  pd.get_dummies(data_val['ED_dispo'])[list(y_train.columns)]



In [ ]:
del data_train, data_test, data_val 

In [ ]:
rng = np.random.RandomState(0)
param_dist = {    'dropout': [0.1,0.2, 0.3],
                  'learning_rate': loguniform.rvs(1e-3, 1e-1, size= 10),
                  'epochs': [1],
                  'batch_size':[128]
                  }
            
dict_parameters = ParameterSampler(param_distributions=param_dist, n_iter= 10, random_state=rng)


In [ ]:
for parameter in dict_parameters:
    print(parameter)
    print('Training')
    with mlflow.start_run(): 
        mlflow_log_parameters(parameter) 
        bert_model = create_bert(parameter['learning_rate'], parameter['dropout'])
        hist = bert_model.fit(x=X_train['Impression'], y=y_train, epochs=parameter['epochs'], batch_size=parameter['batch_size'])
        training_test_mae( mlflow, hist)
        
        print('Testing')
        split_type = 'test'
        y_pred = bert_model.predict(X_test['Impression'])
        accuracy, precision, recall, f1score = evaluate(y_test, y_pred, mlflow, split_type)
        
        print('Validation')
        split_type = 'validation'
        y_pred = bert_model.predict(X_val['Impression'])
        accuracy, precision, recall, f1score = evaluate(y_val, y_pred, mlflow,  split_type)
        
        mlflow.end_run()